# Include lib

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchinfo
import glob
import matplotlib.pyplot as plt
import cv2
import numpy
import os
from os import listdir
import datetime

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models

from PIL import Image, ImageOps

# To use GPU

In [2]:
device = torch.device("cuda")

In [3]:
resnet = torchvision.models.resnet50()
torchinfo.summary(resnet, depth=5, input_size=(128, 3, 64,64), row_settings=["var_names"], verbose=0, col_names=[
"input_size", "output_size", "num_params", "params_percent", "kernel_size", "mult_adds", "trainable"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Param %                   Kernel Shape              Mult-Adds                 Trainable
ResNet (ResNet)                          [128, 3, 64, 64]          [128, 1000]               --                             --                   --                        --                        True
├─Conv2d (conv1)                         [128, 3, 64, 64]          [128, 64, 32, 32]         9,408                       0.04%                   [7, 7]                    1,233,125,376             True
├─BatchNorm2d (bn1)                      [128, 64, 32, 32]         [128, 64, 32, 32]         128                         0.00%                   --                        16,384                    True
├─ReLU (relu)                            [128, 64, 32, 32]         [128, 64, 32, 32]         --                             --                   --                        --              

# Prepareting Dataset

In [4]:
BATCH_SIZE = 32
EPOCH = 20

In [5]:
PATH_TO_TRAIN = ".\pic_data"
PATH_TO_TEST = ".\sy_test"

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Velor139\AppData\Local\Temp\ipykernel_36948\1658379632.py:1: SyntaxWarning: invalid escape sequence '\p'
  PATH_TO_TRAIN = ".\pic_data"
C:\Users\Velor139\AppData\Local\Temp\ipykernel_36948\1658379632.py:2: SyntaxWarning: invalid escape sequence '\s'
  PATH_TO_TEST = ".\sy_test"


In [6]:
labels = dict((name, index) for (index, name) in enumerate(listdir(PATH_TO_TRAIN)))

In [7]:
transformation = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))
])

In [8]:
class Image_Dataset(torch.utils.data.Dataset):
    def __init__(self, image_direcrory, labels, transform=True):
        self.image_direcrory = image_direcrory
        self.labels = labels
        self.transform = transform

        self.paths = list()
        self.labels = list()
        for label in labels: 
            elements = glob.glob(os.path.join(image_direcrory, label, "*.jpg"))
            self.paths.extend(elements)
            self.labels.extend([labels.get(label)] * len(elements))
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        with open(self.paths[index], "rb") as file:
            image = Image.open(file).convert("RGB")
        if self.transform:
            image = transformation(image)
        return image, torch.tensor(self.labels[index], dtype=torch.long)

In [9]:
dataset_train = Image_Dataset(PATH_TO_TRAIN, labels)
dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0) 

# Model

In [10]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False).to(device)
# model.classifier = nn.Sequential(
#     nn.Dropout(0.2),
#     nn.Linear(1280, len(labels))
# ).to(device)

In [11]:
model = models.resnet50(pretrained=True)

# Замена последнего слоя на слой для бинарной классификации
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Один выход для бинарной классификации
model = nn.Sequential(model, nn.Sigmoid()) 
model.to(device)

c:\Users\Velor139\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Velor139\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [12]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

-> для закачки Weights

In [13]:
# model.load_state_dict(torch.load(".\ivan_model_resnet.pth"))

-> обучение

In [14]:
model.train()
epoch_loss = 0
epoch_count = 0
for epoch in range(EPOCH):
    for images, labs in dataloader:
        
        epoch_count += 1
        optimizer.zero_grad()

        images, labs = images.to(device), labs.to(device)

        out = model(images)
        # print(out, labs)
        # print(out.view(-1))
        loss = criterion(out.view(-1), labs.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss
    print(f"epoch = {epoch}  loss = {epoch_loss / epoch_count} ")

epoch = 0  loss = 0.34729841351509094 
epoch = 1  loss = 0.236405149102211 
epoch = 2  loss = 0.23986542224884033 
epoch = 3  loss = 0.21547846496105194 
epoch = 4  loss = 0.19916944205760956 
epoch = 5  loss = 0.18189533054828644 
epoch = 6  loss = 0.1698271781206131 
epoch = 7  loss = 0.1623808592557907 
epoch = 8  loss = 0.15598703920841217 
epoch = 9  loss = 0.14743703603744507 
epoch = 10  loss = 0.13601571321487427 
epoch = 11  loss = 0.12609677016735077 
epoch = 12  loss = 0.12083439528942108 
epoch = 13  loss = 0.11549367755651474 
epoch = 14  loss = 0.1105913296341896 
epoch = 15  loss = 0.10695485025644302 
epoch = 16  loss = 0.10913258045911789 
epoch = 17  loss = 0.10820283740758896 
epoch = 18  loss = 0.10764190554618835 
epoch = 19  loss = 0.10762990266084671 


-> сохранение Weights

In [15]:
torch.save(model.state_dict(), "ivan_model_resnet.pth")

In [16]:
def test_image(model, img_path, transform):
# Load image and apply transformations
    with open(img_path, "rb") as f:
        img = Image.open(f).convert("RGB")  
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Get model output and class probabilities
    output = model(img_tensor)
    # print(output)
    # probs = torch.softmax(output, dim=1)

    # Get predicted class label and class name
    # pred_label = torch.argmax(output, dim=1).item()
    pred_label = output.item()
    # print(output.item())
    # print()
    predicted = 0
    if (pred_label > 0.5):
        predicted = 1 
    # if ()
    class_name = list(labels.keys())[list(labels.values()).index(predicted)]

    # Return predicted label and class probabilities
    return pred_label, class_name

In [17]:
import random
 
number = random.random()  # значение от 0.0 до 1.0

In [18]:
path = PATH_TO_TEST
TP, FP, TN, FN = 0, 0, 0, 0
model.eval()
for i in os.listdir(path):
    # number = random.random()
    # if (number < 0.1):
        pred_label, class_name = test_image(model, os.path.join(path, i), transformation)
        # img = Image.open(PATH_TO_TEST+"\\"+i)
        # fig = plt.figure(figsize=(6, 4))
        # ax = fig.add_subplot()
        # ax.imshow(img)
        
        plt.show()
        if(i.find(class_name) != -1):
            if class_name == "fire":
                TP += 1
            else:
                TN += 1
        else:
            if class_name == "fire":
                FP += 1
            else:
                FN += 1
        # print(f"name {i}, out: {class_name}")
print(f"pres {TP/(TP+FP)}, rec {TP/(TP+FN)} acc {(TP+TN)/(TP+TN+FP+FN)}")

pres 0.9606741573033708, rec 0.9715909090909091 acc 0.9722222222222222
